<a href="https://colab.research.google.com/github/ashibullah/Romanian-Bangla-Sentiment-Analysis-NLP/blob/main/RomanianBanglaSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets --upgrade

In [ ]:
!pip install nltk

In [3]:
import nltk
nltk.download('punkt_tab')  # For tokenizer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
from datasets import load_dataset
from textblob import TextBlob
import re
from nltk.tokenize import word_tokenize
import pandas as pd

import json

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = load_dataset("aplycaebous/BnSentMix" , split = "train")

In [7]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [ ]:
def correct_text(text):
    return str(TextBlob(text).correct())

In [9]:
print(df[20])

{'Sentence': 'Vloi bt cash out korte 15 takar jaigai 18.50 paisa nei', 'Label': 3}


In [11]:
# Create DataFrame
df = pd.DataFrame(df)
# Convert 'Sentence' column to lowercase
df['Sentence'] = df['Sentence'].str.lower()
df

,Sentence,Label
0,youtube ar volg gula boring hoia jaitase din d...,3
1,"your video making, camera work is really good....",3
2,you made me nostalgic. college life a ei dokan...,3
3,"workshop ta engaging, but resources ta insuffi...",3
4,win hoy nay but anek valo khelecu,3
...,...,...
20010,1 march use kortasi pocof5 kono problem nai ga...,0
20011,1 day beshi stay kora jae na,0
20012,1 boro na 2 boro tushar vai er mon boro,0
20013,1 boro na 2 boro sam vai er mon boro,0


In [12]:
# remove urls
df['Sentence'] = df['Sentence'].apply(clean_text)
df

,Sentence,Label
0,youtube ar volg gula boring hoia jaitase din d...,3
1,your video making camera work is really good i...,3
2,you made me nostalgic college life a ei dokan ...,3
3,workshop ta engaging but resources ta insuffic...,3
4,win hoy nay but anek valo khelecu,3
...,...,...
20010,1 march use kortasi pocof5 kono problem nai ga...,0
20011,1 day beshi stay kora jae na,0
20012,1 boro na 2 boro tushar vai er mon boro,0
20013,1 boro na 2 boro sam vai er mon boro,0


In [ ]:
# df = df['Sentence'].apply(correct_text)
# avoiding this

In [13]:
dict_path = '/content/drive/MyDrive/Colab Notebooks/RomanianBanglaUtilities/normalization_dict.json'
with open(dict_path, 'r', encoding='utf-8') as f:
    normalization_dict = json.load(f)

print(f"Loaded normalization dictionary with {len(normalization_dict)} entries")


# Function to normalize text using your dictionary
def normalize_text(text, norm_dict):
    tokens = text.split()
    reverse_map = {}
    for std_word, variants in norm_dict.items():
        reverse_map[std_word] = std_word
        for var in variants:
            reverse_map[var] = std_word
    normalized_tokens = [reverse_map.get(token.lower(), token.lower()) for token in tokens]
    return ' '.join(normalized_tokens)



Loaded normalization dictionary with 58 entries


In [14]:
df['Sentence'] = df['Sentence'].apply(lambda x: normalize_text(x, normalization_dict))

In [15]:
abbrev_path = '/content/drive/MyDrive/Colab Notebooks/RomanianBanglaUtilities/abbreviation.json'
with open(abbrev_path, 'r', encoding='utf-8') as f:
    abbreviation_dict = json.load(f)

print(f"Loaded abbreviation dictionary with {len(abbreviation_dict)} entries")

# Function to normalize text using abbreviation dictionary
def normalize_abbreviations(text, abbr_dict):
    tokens = text.split()
    normalized_tokens = [abbr_dict.get(token.lower(), token) for token in tokens]
    return ' '.join(normalized_tokens)

Loaded abbreviation dictionary with 72 entries


In [16]:
df['Sentence'] = df['Sentence'].apply(lambda x: normalize_abbreviations(x, abbreviation_dict))

In [17]:
df['Tokens'] = df['Sentence'].apply(word_tokenize)

df

,Sentence,Label,Tokens
0,youtube ar volg gula boring hoye jaitase din d...,3,"[youtube, ar, volg, gula, boring, hoye, jaitas..."
1,your video making camera work is really good i...,3,"[your, video, making, camera, work, is, really..."
2,you made me nostalgic college life a ei dokan ...,3,"[you, made, me, nostalgic, college, life, a, e..."
3,workshop ta engaging but resources ta insuffic...,3,"[workshop, ta, engaging, but, resources, ta, i..."
4,win hoy nay but onek bhalo khelecu,3,"[win, hoy, nay, but, onek, bhalo, khelecu]"
...,...,...,...
20010,1 march use kortasi pocof5 kono problem nai ga...,0,"[1, march, use, kortasi, pocof5, kono, problem..."
20011,1 day beshi stay kora jae na,0,"[1, day, beshi, stay, kora, jae, na]"
20012,1 boro na 2 boro tushar bhai er mon boro,0,"[1, boro, na, 2, boro, tushar, bhai, er, mon, ..."
20013,1 boro na 2 boro sam bhai er mon boro,0,"[1, boro, na, 2, boro, sam, bhai, er, mon, boro]"


from matplotlib import pyplot as plt
df['Label'].plot(kind='hist', bins=20, title='Label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Label'].plot(kind='line', figsize=(8, 4), title='Label')
plt.gca().spines[['top', 'right']].set_visible(False)